In [1]:
# import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# To create linear regression model
from sklearn.linear_model import LinearRegression

In [2]:
# Load data
df_sales = pd.read_csv('Walmart_Store_sales.csv')
df_sales.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [3]:
df_sales.shape

(6435, 8)

In [4]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [5]:
df_sales.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


## Basic Statistics tasks

#### Which store has maximum sales?

In [6]:
# Let's first find the store with maximum total sales across the entire period
# Get the sum of sales for each store using groupby function
store_sales = df_sales.groupby('Store')['Weekly_Sales'].sum()

# Get the max sales revenue among the stores
max_sales = store_sales.max()

# find the store with maximum sales
store_max_sales = store_sales[store_sales == max_sales].reset_index()['Store']
print("Store {} has the maximum sales with a total sales revenue of {:.2f}.".format(int(store_max_sales),max_sales))

Store 20 has the maximum sales with a total sales revenue of 301397792.46.


In [7]:
# Now, let's find the store which achieved the maximum weekly sales
# Try with an alternative method: sort max weekly sales of each store in descending order
store_weekly_max = df_sales.groupby('Store')['Weekly_Sales'].max()
store_weekly_max.sort_values(ascending=False).head()

Store
14    3818686.45
20    3766687.43
10    3749057.69
4     3676388.98
13    3595903.20
Name: Weekly_Sales, dtype: float64

<font color=darkblue>
From the outcome above we find that Store 14 has the maximum weekly sales.

#### Which store has maximum standard deviation? And what is its coefficient of mean to standard deviation (coefficient of variation)?

In [8]:
# Derive standard deviation of each store
store_std = df_sales.groupby('Store')['Weekly_Sales'].std()

# Find the max standard deviation
max_std = store_std.max()

# find the store with maximum standard deviation
store_max__std = store_std[store_std == max_std].reset_index()['Store']
print("Store {} has the maximum standard deviation, which is {:.2f}.".format(int(store_max__std),max_std))

# Now let's get the mean of each store
store_sales_mean = df_sales.groupby('Store')['Weekly_Sales'].mean()

# compute its coefficient of mean to standard deviation
coef_mean_std = np.divide(max_std,store_sales_mean[int(store_max__std)])
print(f"The coefficient of mean to standard deviation for store {int(store_max__std)} =",round(coef_mean_std,4))

Store 14 has the maximum standard deviation, which is 317569.95.
The coefficient of mean to standard deviation for store 14 = 0.1571


#### Which store/s has good quarterly growth rate in Q3’2012?

In [9]:
# Covert date column into datetime format
df_sales['Date'] = pd.to_datetime(df_sales['Date'])

# Firstly, we find Q3 sales for every store in 2012
df_Q3_2012 = df_sales[(df_sales['Date']>='2012-07-01') & (df_sales['Date']<='2012-09-30')]
df_Q3_2012_sales = df_Q3_2012.groupby('Store')['Weekly_Sales'].sum()

# Secondly, we find Q2 sales for every store
df_Q2_2012 = df_sales[(df_sales['Date']>='2012-04-01') & (df_sales['Date']<='2012-06-30')]
df_Q2_2012_sales = df_Q2_2012.groupby('Store')['Weekly_Sales'].sum()

# Thirdly, we find the sales growth from Q2 to Q3 for each store
quarterly_growth = np.subtract(df_Q3_2012_sales,df_Q2_2012_sales)

# Forthly, calculate quarterly growth rate for each store
quarterly_growth_rate = np.divide(quarterly_growth,df_Q2_2012_sales)

# Forthly, we sort the stores with quarterly growth rate in descending order
top_5_growths = quarterly_growth_rate.sort_values(ascending=False).head()

# Now, let's view the top 5 stores with highest growth 
print(top_5_growths)

Store
16   -0.027893
7    -0.038247
35   -0.046631
26   -0.060576
39   -0.063969
Name: Weekly_Sales, dtype: float64


<font color=darkblue>
    
- Notice that the top performing store, store 16, is at negative quarterly growth rate of -0.027893, which means all the stores experienced a drop in sales.
- Hence, we can conclude that none of the stores has good quarterly growth rate in Q3’2012.

#### Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together:

In [10]:
# calculate the mean sales in non-holiday season
non_holiday_mean_sales = df_sales[df_sales['Holiday_Flag']==0]['Weekly_Sales'].mean()
non_holiday_mean_sales

1041256.3802088564

In [11]:
# calculate the mean sales for each holiday event
df_holiday_sales = df_sales[df_sales['Holiday_Flag']==1].groupby('Date')['Weekly_Sales'].mean()

In [12]:
# find the holidays with higher sales than mean sales in non-holiday season
good_holiday_sales = df_holiday_sales[df_holiday_sales>non_holiday_mean_sales]
print("The holidays with sales higher than the mean sales in non-holiday season are: \n", good_holiday_sales)

The holidays with sales higher than the mean sales in non-holiday season are: 
 Date
2010-11-26    1.462689e+06
2010-12-02    1.074148e+06
2011-11-02    1.051915e+06
2011-11-25    1.479858e+06
2012-07-09    1.074001e+06
2012-10-02    1.111320e+06
Name: Weekly_Sales, dtype: float64


#### Provide a monthly and semester view of sales in units and give insights:

In [13]:
# create corresponding columns of month and year
df_sales['Month'] = pd.DatetimeIndex(df_sales['Date']).month
df_sales['Year'] = pd.DatetimeIndex(df_sales['Date']).year

In [14]:
df_sales.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,5,2010
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,12,2010
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2,2010
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2,2010
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106,5,2010


In [15]:
# Let's find monthly sales
monthly_sales = df_sales.groupby(['Year','Month'])[['Weekly_Sales']].sum()
monthly_sales = monthly_sales.rename(columns={'Weekly_Sales':'Total Monthly Sales'})
monthly_sales

Total Monthly Sales
Year Month                     
2010 1             4.223988e+07
     2             1.915869e+08
     3             1.862262e+08
     4             1.838118e+08
     5             2.806119e+08
     6             1.424361e+08
     7             1.842664e+08
     8             1.845381e+08
     9             1.797041e+08
     10            2.311201e+08
     11            1.587731e+08
     12            3.235716e+08
2011 1             2.119657e+08
     2             1.876092e+08
     3             1.365205e+08
     4             2.789693e+08
     5             1.828017e+08
     6             1.401936e+08
     7             2.244611e+08
     8             1.880810e+08
     9             2.310323e+08
     10            1.837193e+08
     11            2.534703e+08
     12            2.293760e+08
2012 1             1.722207e+08
     2             1.428296e+08
     3             2.307397e+08
     4             1.825428e+08
     5             1.422830e+08
     6             2.923883e+08
     7             1.845865e+08
     8             1.916126e+08
     9             1.797959e+08
     10            1.880794e+08
     11            4.692588e+07
     12            4.612851e+07

<font color=darkblue>

- Notice that the monthly sales had been above 140mil across the period except Jan-2010, Nov-2012, and Dec-2012;
- Dec 2020 achived the highest monthly sales and it was the only month with sales above 300mil during the 3 years.

In [16]:
# create a corresponding column of Semester (semi-annually)
# set value 1 being first half of the year and 2 being sedond half of the year
df_sales['Semester'] = pd.DatetimeIndex(df_sales['Date']).quarter
df_sales['Semester'] = df_sales['Semester'].replace([1,2],1)
df_sales['Semester'] = df_sales['Semester'].replace([3,4],2)
df_sales.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year,Semester
0,1,2010-05-02,1643690.90,0,42.31,2.572,211.096358,8.106,5,2010,1
1,1,2010-12-02,1641957.44,1,38.51,2.548,211.242170,8.106,12,2010,2
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2,2010,1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2,2010,1
4,1,2010-05-03,1554806.68,0,46.50,2.625,211.350143,8.106,5,2010,1


In [17]:
# Now let's find quarterly sales
quarterly_sales = df_sales.groupby(['Year','Semester'])[['Weekly_Sales']].sum()
quarterly_sales = quarterly_sales.rename(columns={'Weekly_Sales':'Total Semester Sales'})
quarterly_sales

Total Semester Sales
Year Semester                      
2010 1                 1.026913e+09
     2                 1.261973e+09
2011 1                 1.138060e+09
     2                 1.310140e+09
2012 1                 1.163004e+09
     2                 8.371288e+08

<font color=darkblue>
    
- Total semester sales from 2010 to first half of 2012 remained stable at above 1 billion;
- In second half of 2012 , the total semester sales fell drastically to about 837 million.

## Statistical Model

#### For Store 1 – Build  prediction models to forecast demand
Linear Regression – Utilize variables like date and restructure dates as 1 for 5 Feb 2010 (starting from the earliest date in order). Hypothesize if CPI, unemployment, and fuel price have any impact on sales.

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.formula.api as smf

In [19]:
# Let's create a dataframe for Store 1 and order it by date 
df_store1 = df_sales[df_sales['Store']==1].sort_values(by='Date').reset_index()

# discard unrequired columns including 'Store' because the data is all from Store 1
df_store1 = df_store1.drop(['index', 'Store','Month','Year','Semester'], axis=1)
df_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,2010-01-10,1453329.50,0,71.89,2.603,211.671989,7.838
1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808
2,2010-02-07,1492418.14,0,80.91,2.669,211.223533,7.787
3,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
4,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106


In [20]:
# restructure dates as 1 for 5 Feb 2010 (starting from the earliest date in order)
# this is to convert datetime type to int for regression model yet retaining its time sequence 
from sklearn.preprocessing import LabelEncoder
LabelEncoder = LabelEncoder()
df_store1['Date'] = LabelEncoder.fit_transform(df_store1['Date']) + 1
df_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,1453329.50,0,71.89,2.603,211.671989,7.838
1,2,1594968.28,0,62.27,2.719,210.820450,7.808
2,3,1492418.14,0,80.91,2.669,211.223533,7.787
3,4,1611968.17,0,39.93,2.514,211.289143,8.106
4,5,1409727.59,0,46.63,2.561,211.319643,8.106


In [21]:
df_store1.shape

(143, 7)

- Hypothesize if CPI, unemployment, and fuel price have any impact on sales.

- Null Hypothesis: CPI, unemployment, and fuel price have no impact on sales

- Alternative Hypothesis: CPI, unemployment, and fuel price have impact on sales

In [22]:
lin_model = smf.ols(formula=' Weekly_Sales ~ Fuel_Price + CPI + Unemployment', data=df_store1).fit()

In [23]:
# find p-value
lin_model.pvalues < 0.05

Intercept        True
Fuel_Price      False
CPI              True
Unemployment     True
dtype: bool

<font color=darkblue>
    
- We reject the null hypothesis of CPI and unemployment haaving no impact on sales, and accept the null hypothesis of fuel_price having no impact on sales.
- In other word, at 5% significance, we are confident to say that fuel_price has an impact on sales, while CPI and unemployment do not have.
- Therefore, we may drop CPI and unemployment variable for the Linear Regression.

In [24]:
# generate train and test dataset
X = df_store1.drop(['Weekly_Sales','CPI', 'Unemployment'],axis=1)
y = df_store1['Weekly_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=21)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(114, 4)
(29, 4)
(114,)
(29,)


In [26]:
# fit model
LinReg = LinearRegression()
LinReg.fit(X_train,y_train)

LinearRegression()

In [27]:
print("Intercept:", LinReg.intercept_)
print("Coefficients: ",LinReg.coef_)

Intercept: 1627762.4690774493
Coefficients:  [   799.97271759 118429.08345314  -2238.99894812   5218.03042394]


In [28]:
# define the equation of regression 
print('Weekly Sales =',LinReg.intercept_,end=' ')
for i in range(len(LinReg.coef_)):
    print('+ (',LinReg.coef_[i],')*',X.columns[i],end=' ')

Weekly Sales = 1627762.4690774493 + ( 799.9727175896027 )* Date + ( 118429.0834531389 )* Holiday_Flag + ( -2238.9989481231687 )* Temperature + ( 5218.030423944468 )* Fuel_Price 

In [29]:
# get predictions
y_pred = LinReg.predict(X_test)

In [30]:
# Calculate accuracy score
print("Coefficient of determination  of the prediction, R^2 score =",metrics.r2_score(y_test,y_pred).round(4))

# calculate MAE
print("Mean Absolute Error score =",metrics.mean_absolute_error(y_test,y_pred).round(4))

# Calculate RMSE
print("Root Mean Square Error score =",np.sqrt(metrics.mean_squared_error(y_test,y_pred)).round(4))



Coefficient of determination  of the prediction, R^2 score = 0.0894
Mean Absolute Error score = 103355.3359
Root Mean Square Error score = 136205.2314


<font color=darkblue>
    
- Notice that this model only has a R^2 score of 0.0894, which means the variation in sales isn't well-explained by any of the independent variables.
- This models cocmes with large MAE and RMSE, it seems like isn't a good model to predict.

#### Let's now create a new model with a new variable of changing date to day:

In [31]:
# duplicate df_store1 and transform date labels back to original encoding
df2_store1 = df_store1.copy()
df2_store1['Date'] = LabelEncoder.inverse_transform(df2_store1['Date']-1)
df2_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,2010-01-10,1453329.50,0,71.89,2.603,211.671989,7.838
1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808
2,2010-02-07,1492418.14,0,80.91,2.669,211.223533,7.787
3,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
4,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106


In [32]:
# Change dates into days by creating new variable
df2_store1['Day'] = pd.DatetimeIndex(df2_store1['Date']).day_name()
df2_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day
0,2010-01-10,1453329.50,0,71.89,2.603,211.671989,7.838,Sunday
1,2010-02-04,1594968.28,0,62.27,2.719,210.820450,7.808,Thursday
2,2010-02-07,1492418.14,0,80.91,2.669,211.223533,7.787,Sunday
3,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,Friday
4,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,Friday


In [33]:
# Now, restructure 'Date' back to int again
df2_store1['Date'] = LabelEncoder.fit_transform(df2_store1['Date']) + 1
df2_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day
0,1,1453329.50,0,71.89,2.603,211.671989,7.838,Sunday
1,2,1594968.28,0,62.27,2.719,210.820450,7.808,Thursday
2,3,1492418.14,0,80.91,2.669,211.223533,7.787,Sunday
3,4,1611968.17,0,39.93,2.514,211.289143,8.106,Friday
4,5,1409727.59,0,46.63,2.561,211.319643,8.106,Friday


In [34]:
# Apply one-hot encoding on the column 'Day'
df2_store1 = pd.get_dummies(df2_store1, columns=['Day'])
df2_store1.head()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,1,1453329.50,0,71.89,2.603,211.671989,7.838,0,0,0,1,0,0,0
1,2,1594968.28,0,62.27,2.719,210.820450,7.808,0,0,0,0,1,0,0
2,3,1492418.14,0,80.91,2.669,211.223533,7.787,0,0,0,1,0,0,0
3,4,1611968.17,0,39.93,2.514,211.289143,8.106,1,0,0,0,0,0,0
4,5,1409727.59,0,46.63,2.561,211.319643,8.106,1,0,0,0,0,0,0


In [35]:
# generate new train and test dataset
X2 = df2_store1.drop(['Weekly_Sales','CPI', 'Unemployment', 'Day_Friday'],axis=1)
y2 = df2_store1['Weekly_Sales']

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,test_size=0.2, random_state=21)

In [36]:
print(X2_train.shape)
print(X2_test.shape)
print(y2_train.shape)
print(y2_test.shape)

(114, 10)
(29, 10)
(114,)
(29,)


In [37]:
# fit model
LinReg2 = LinearRegression()
LinReg2.fit(X2_train,y2_train)

LinearRegression()

In [38]:
print("Intercept:", LinReg2.intercept_)
print("Coefficients: ",LinReg2.coef_)

Intercept: 1649043.4769861312
Coefficients:  [   818.41248285 118420.12583407  -2402.20373085  -7750.91544429
 124723.28510359  10133.8378857   96859.17164784 123215.79032429
  83877.23026745   6528.9229404 ]


In [39]:
# get predictions
y2_pred = LinReg2.predict(X2_test)

In [40]:
# Calculate accuracy score
print("Coefficient of determination  of the prediction, R^2 score =",metrics.r2_score(y2_test,y2_pred).round(4))

# calculate MAE
print("Mean Absolute Error score =",metrics.mean_absolute_error(y2_test,y2_pred).round(4))

# Calculate RMSE
print("Root Mean Square Error score =",np.sqrt(metrics.mean_squared_error(y2_test,y2_pred)).round(4))

Coefficient of determination  of the prediction, R^2 score = 0.0501
Mean Absolute Error score = 109055.2235
Root Mean Square Error score = 139113.9177


<font color=darkblue>
    
- R^2 score of second model is lesser than the first model, and it comes with larger MAE and RMSE scores.
- Hence, the first model gives the better accuracy than the second model.